In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RS").getOrCreate()
df = spark.read.csv("/FileStore/tables/articles1.csv", inferSchema=True, header=True)
data2 = spark.read.csv("/FileStore/tables/articles2.csv", inferSchema=True, header=True)
data3 = spark.read.csv("/FileStore/tables/articles3.csv", inferSchema=True, header=True)

In [2]:
df = df.union(data2)

In [3]:
df = df.union(data3)

In [4]:
df.show()

+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
_c0| id| title| publication| author| date| year|month| url| content|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
 0|17283|House Republicans...|New York Times| Carl Hulse|2016-12-31|2016.0| 12.0|null|WASHINGTON — C...|
 1|17284|Rift Between Offi...|New York Times|Benjamin Mueller ...|2017-06-19|2017.0| 6.0|null|After the bullet ...|
 2|17285|Tyrus Wong, ‘Bamb...|New York Times| Margalit Fox|2017-01-06|2017.0| 1.0|null|When Walt Disney’...|
 3|17286|Among Deaths in 2...|New York Times| William McDonald|2017-04-10|2017.0| 4.0|null|Death may be the ...|
 4|17287|Kim Jong-un Says ...|New York Times| Choe Sang-Hun|2017-01-02|2017.0| 1.0|null|SEOUL, South Kore...|
 5|17288|Sick With a Cold,...|New York Times| Sewell Chan|2017-01-02|2017.0| 1.0|null|LONDON — Queen...|
 6|17289|Taiwan’s Presiden...|New York Times| Javier C. Hernández|2017-01-02|2017.0| 1.0|null|BEIJING — Pres...|
 7|17290|After ‘The Bigges...|New York Times| Gina Kolata|2017-02-08|2017.0| 2.0|null|Danny Cahill stoo...|
 8|17291|First, a Mixtape....|New York Times| Katherine Rosman|2016-12-31|2016.0| 12.0|null|Just how is Hil...|
 9|17292|Calling on Angels...|New York Times| Andy Newman|2016-12-31|2016.0| 12.0|null|Angels are everyw...|
 10|17293|Weak Federal Powe...|New York Times| Justin Gillis|2017-01-03|2017.0| 1.0|null|With Donald J. Tr...|
 11|17294|Can Carbon Captur...|New York Times| John Schwartz|2017-01-05|2017.0| 1.0|null|THOMPSONS, Tex. ...|
 12|17295|Mar-a-Lago, the F...|New York Times| Maggie Haberman|2017-01-02|2017.0| 1.0|null|WEST PALM BEACH, ...|
 13|17296|How to form healt...|New York Times| Charles Duhigg|2017-01-02|2017.0| 1.0|null|This article is p...|
 14|17297|Turning Your Vaca...|New York Times|Stephanie Rosenbloom|2017-04-14|2017.0| 4.0|null|It’s the season f...|
 15|17298|As Second Avenue ...|New York Times| Emma G. Fitzsimmons|2017-01-02|2017.0| 1.0|null|Finally. The Seco...|
 16|17300|Dylann Roof Himse...|New York Times|Kevin Sack and Al...|2017-01-02|2017.0| 1.0|null| pages into the ...|
 17|17301|Modi’s Cash Ban B...|New York Times| Geeta Anand|2017-01-02|2017.0| 1.0|null|MUMBAI, India — ...|
 18|17302|Suicide Bombing i...|New York Times|The Associated Press|2017-01-03|2017.0| 1.0|null|BAGHDAD — A su...|
 19|17303|Fecal Pollution T...|New York Times| Brett Cole|2017-01-03|2017.0| 1.0|null|SYDNEY, Australia...|
+---+-----+--------------------+--------------+--------------------+----------+------+-----+----+--------------------+
only showing top 20 rows

In [5]:
df = df.select("id", "title", "content")

In [6]:
df.show()

+-----+--------------------+--------------------+
 id| title| content|
+-----+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|
17284|Rift Between Offi...|After the bullet ...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|
17286|Among Deaths in 2...|Death may be the ...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|
17288|Sick With a Cold,...|LONDON — Queen...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|
17290|After ‘The Bigges...|Danny Cahill stoo...|
17291|First, a Mixtape....|Just how is Hil...|
17292|Calling on Angels...|Angels are everyw...|
17293|Weak Federal Powe...|With Donald J. Tr...|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|
17296|How to form healt...|This article is p...|
17297|Turning Your Vaca...|It’s the season f...|
17298|As Second Avenue ...|Finally. The Seco...|
17300|Dylann Roof Himse...| pages into the ...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|
17302|Suicide Bombing i...|BAGHDAD — A su...|
17303|Fecal Pollution T...|SYDNEY, Australia...|
+-----+--------------------+--------------------+
only showing top 20 rows

In [7]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [8]:
from pyspark.sql.functions import col, udf

In [9]:
from pyspark.sql.types import IntegerType

In [10]:
tokenizer = Tokenizer(inputCol='content', outputCol = 'words')
regex_tokenizer = RegexTokenizer(inputCol='content', outputCol='words', pattern=r'[^a-zA-Z0-9]+')

In [11]:
count_tokens = udf(lambda words:len(words), IntegerType())

In [12]:
tokenized = tokenizer.transform(df)

In [13]:
tokenized.show()

+-----+--------------------+--------------------+--------------------+
 id| title| content| words|
+-----+--------------------+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|[washington, , —,...|
17284|Rift Between Offi...|After the bullet ...|[after, the, bull...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|[when, walt, disn...|
17286|Among Deaths in 2...|Death may be the ...|[death, may, be, ...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|[seoul,, south, k...|
17288|Sick With a Cold,...|LONDON — Queen...|[london, , —, , ,...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|[beijing, , —, , ...|
17290|After ‘The Bigges...|Danny Cahill stoo...|[danny, cahill, s...|
17291|First, a Mixtape....|Just how is Hil...|[just, how, , , i...|
17292|Calling on Angels...|Angels are everyw...|[angels, are, eve...|
17293|Weak Federal Powe...|With Donald J. Tr...|[with, donald, j....|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|[thompsons,, tex....|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|[west, palm, beac...|
17296|How to form healt...|This article is p...|[this, article, i...|
17297|Turning Your Vaca...|It’s the season f...|[it’s, the, seaso...|
17298|As Second Avenue ...|Finally. The Seco...|[finally., the, s...|
17300|Dylann Roof Himse...| pages into the ...|[, , pages, into,...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|[mumbai,, india, ...|
17302|Suicide Bombing i...|BAGHDAD — A su...|[baghdad, , —, , ...|
17303|Fecal Pollution T...|SYDNEY, Australia...|[sydney,, austral...|
+-----+--------------------+--------------------+--------------------+
only showing top 20 rows

In [14]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+-----+--------------------+--------------------+--------------------+------+
 id| title| content| words|tokens|
+-----+--------------------+--------------------+--------------------+------+
17283|House Republicans...|WASHINGTON — C...|[washington, , —,...| 920|
17284|Rift Between Offi...|After the bullet ...|[after, the, bull...| 4905|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|[when, walt, disn...| 2521|
17286|Among Deaths in 2...|Death may be the ...|[death, may, be, ...| 2212|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|[seoul,, south, k...| 741|
17288|Sick With a Cold,...|LONDON — Queen...|[london, , —, , ,...| 168|
17289|Taiwan’s Presiden...|BEIJING — Pres...|[beijing, , —, , ...| 582|
17290|After ‘The Bigges...|Danny Cahill stoo...|[danny, cahill, s...| 3264|
17291|First, a Mixtape....|Just how is Hil...|[just, how, , , i...| 1643|
17292|Calling on Angels...|Angels are everyw...|[angels, are, eve...| 845|
17293|Weak Federal Powe...|With Donald J. Tr...|[with, donald, j....| 1208|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|[thompsons,, tex....| 1864|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|[west, palm, beac...| 1403|
17296|How to form healt...|This article is p...|[this, article, i...| 695|
17297|Turning Your Vaca...|It’s the season f...|[it’s, the, seaso...| 1858|
17298|As Second Avenue ...|Finally. The Seco...|[finally., the, s...| 1403|
17300|Dylann Roof Himse...| pages into the ...|[, , pages, into,...| 1627|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|[mumbai,, india, ...| 1582|
17302|Suicide Bombing i...|BAGHDAD — A su...|[baghdad, , —, , ...| 574|
17303|Fecal Pollution T...|SYDNEY, Australia...|[sydney,, austral...| 299|
+-----+--------------------+--------------------+--------------------+------+
only showing top 20 rows

In [15]:
regex_tokenizer= RegexTokenizer(inputCol='content', outputCol='words',pattern='[^a-zA-Z0-9]+').setMinTokenLength(2)
df=df.fillna("unknown", subset=["content","title"])
rg_tokenized = regex_tokenizer.transform(df)

In [16]:
rg_tokenized.show()

+-----+--------------------+--------------------+--------------------+
 id| title| content| words|
+-----+--------------------+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|[washington, cong...|
17284|Rift Between Offi...|After the bullet ...|[after, the, bull...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|[when, walt, disn...|
17286|Among Deaths in 2...|Death may be the ...|[death, may, be, ...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|[seoul, south, ko...|
17288|Sick With a Cold,...|LONDON — Queen...|[london, queen, e...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|[beijing, preside...|
17290|After ‘The Bigges...|Danny Cahill stoo...|[danny, cahill, s...|
17291|First, a Mixtape....|Just how is Hil...|[just, how, is, h...|
17292|Calling on Angels...|Angels are everyw...|[angels, are, eve...|
17293|Weak Federal Powe...|With Donald J. Tr...|[with, donald, tr...|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|[thompsons, tex, ...|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|[west, palm, beac...|
17296|How to form healt...|This article is p...|[this, article, i...|
17297|Turning Your Vaca...|It’s the season f...|[it, the, season,...|
17298|As Second Avenue ...|Finally. The Seco...|[finally, the, se...|
17300|Dylann Roof Himse...| pages into the ...|[pages, into, the...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|[mumbai, india, i...|
17302|Suicide Bombing i...|BAGHDAD — A su...|[baghdad, suicide...|
17303|Fecal Pollution T...|SYDNEY, Australia...|[sydney, australi...|
+-----+--------------------+--------------------+--------------------+
only showing top 20 rows

In [17]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+-----+--------------------+--------------------+--------------------+------+
 id| title| content| words|tokens|
+-----+--------------------+--------------------+--------------------+------+
17283|House Republicans...|WASHINGTON — C...|[washington, cong...| 851|
17284|Rift Between Offi...|After the bullet ...|[after, the, bull...| 4587|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|[when, walt, disn...| 2270|
17286|Among Deaths in 2...|Death may be the ...|[death, may, be, ...| 2038|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|[seoul, south, ko...| 666|
17288|Sick With a Cold,...|LONDON — Queen...|[london, queen, e...| 151|
17289|Taiwan’s Presiden...|BEIJING — Pres...|[beijing, preside...| 553|
17290|After ‘The Bigges...|Danny Cahill stoo...|[danny, cahill, s...| 3038|
17291|First, a Mixtape....|Just how is Hil...|[just, how, is, h...| 1530|
17292|Calling on Angels...|Angels are everyw...|[angels, are, eve...| 783|
17293|Weak Federal Powe...|With Donald J. Tr...|[with, donald, tr...| 1140|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|[thompsons, tex, ...| 1727|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|[west, palm, beac...| 1257|
17296|How to form healt...|This article is p...|[this, article, i...| 628|
17297|Turning Your Vaca...|It’s the season f...|[it, the, season,...| 1705|
17298|As Second Avenue ...|Finally. The Seco...|[finally, the, se...| 1294|
17300|Dylann Roof Himse...| pages into the ...|[pages, into, the...| 1521|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|[mumbai, india, i...| 1502|
17302|Suicide Bombing i...|BAGHDAD — A su...|[baghdad, suicide...| 535|
17303|Fecal Pollution T...|SYDNEY, Australia...|[sydney, australi...| 286|
+-----+--------------------+--------------------+--------------------+------+
only showing top 20 rows

In [18]:
from pyspark.ml.feature import StopWordsRemover

In [19]:
remover = StopWordsRemover(inputCol='words', outputCol='filtered')

In [20]:
remover.transform(rg_tokenized).show()

+-----+--------------------+--------------------+--------------------+--------------------+
 id| title| content| words| filtered|
+-----+--------------------+--------------------+--------------------+--------------------+
17283|House Republicans...|WASHINGTON — C...|[washington, cong...|[washington, cong...|
17284|Rift Between Offi...|After the bullet ...|[after, the, bull...|[bullet, shells, ...|
17285|Tyrus Wong, ‘Bamb...|When Walt Disney’...|[when, walt, disn...|[walt, disney, ba...|
17286|Among Deaths in 2...|Death may be the ...|[death, may, be, ...|[death, may, grea...|
17287|Kim Jong-un Says ...|SEOUL, South Kore...|[seoul, south, ko...|[seoul, south, ko...|
17288|Sick With a Cold,...|LONDON — Queen...|[london, queen, e...|[london, queen, e...|
17289|Taiwan’s Presiden...|BEIJING — Pres...|[beijing, preside...|[beijing, preside...|
17290|After ‘The Bigges...|Danny Cahill stoo...|[danny, cahill, s...|[danny, cahill, s...|
17291|First, a Mixtape....|Just how is Hil...|[just, how, is, h...|[hillary, kerr, f...|
17292|Calling on Angels...|Angels are everyw...|[angels, are, eve...|[angels, everywhe...|
17293|Weak Federal Powe...|With Donald J. Tr...|[with, donald, tr...|[donald, trump, t...|
17294|Can Carbon Captur...|THOMPSONS, Tex. ...|[thompsons, tex, ...|[thompsons, tex, ...|
17295|Mar-a-Lago, the F...|WEST PALM BEACH, ...|[west, palm, beac...|[west, palm, beac...|
17296|How to form healt...|This article is p...|[this, article, i...|[article, part, s...|
17297|Turning Your Vaca...|It’s the season f...|[it, the, season,...|[season, family, ...|
17298|As Second Avenue ...|Finally. The Seco...|[finally, the, se...|[finally, second,...|
17300|Dylann Roof Himse...| pages into the ...|[pages, into, the...|[pages, journal, ...|
17301|Modi’s Cash Ban B...|MUMBAI, India — ...|[mumbai, india, i...|[mumbai, india, b...|
17302|Suicide Bombing i...|BAGHDAD — A su...|[baghdad, suicide...|[baghdad, suicide...|
17303|Fecal Pollution T...|SYDNEY, Australia...|[sydney, australi...|[sydney, australi...|
+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [21]:
final_df = remover.transform(rg_tokenized).select(col('id'), col('title'), col('filtered').alias('words'))

In [22]:
final_df.show()

+-----+--------------------+--------------------+
 id| title| words|
+-----+--------------------+--------------------+
17283|House Republicans...|[washington, cong...|
17284|Rift Between Offi...|[bullet, shells, ...|
17285|Tyrus Wong, ‘Bamb...|[walt, disney, ba...|
17286|Among Deaths in 2...|[death, may, grea...|
17287|Kim Jong-un Says ...|[seoul, south, ko...|
17288|Sick With a Cold,...|[london, queen, e...|
17289|Taiwan’s Presiden...|[beijing, preside...|
17290|After ‘The Bigges...|[danny, cahill, s...|
17291|First, a Mixtape....|[hillary, kerr, f...|
17292|Calling on Angels...|[angels, everywhe...|
17293|Weak Federal Powe...|[donald, trump, t...|
17294|Can Carbon Captur...|[thompsons, tex, ...|
17295|Mar-a-Lago, the F...|[west, palm, beac...|
17296|How to form healt...|[article, part, s...|
17297|Turning Your Vaca...|[season, family, ...|
17298|As Second Avenue ...|[finally, second,...|
17300|Dylann Roof Himse...|[pages, journal, ...|
17301|Modi’s Cash Ban B...|[mumbai, india, b...|
17302|Suicide Bombing i...|[baghdad, suicide...|
17303|Fecal Pollution T...|[sydney, australi...|
+-----+--------------------+--------------------+
only showing top 20 rows

In [23]:
from pyspark.ml.feature import HashingTF, IDF

In [24]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [25]:
featurized_data = hashing_tf.transform(final_df)
featurized_data.cache

Out[ 104 ]: <bound method DataFrame.cache of DataFrame[id: string, title: string, words: array<string>, rawFeatures: vector]>

In [26]:
featurized_data.show()

+-----+--------------------+--------------------+--------------------+
 id| title| words| rawFeatures|
+-----+--------------------+--------------------+--------------------+
17283|House Republicans...|[washington, cong...|(262144,[619,2326...|
17284|Rift Between Offi...|[bullet, shells, ...|(262144,[14,343,1...|
17285|Tyrus Wong, ‘Bamb...|[walt, disney, ba...|(262144,[14,535,5...|
17286|Among Deaths in 2...|[death, may, grea...|(262144,[75,535,7...|
17287|Kim Jong-un Says ...|[seoul, south, ko...|(262144,[2472,328...|
17288|Sick With a Cold,...|[london, queen, e...|(262144,[2564,538...|
17289|Taiwan’s Presiden...|[beijing, preside...|(262144,[789,1232...|
17290|After ‘The Bigges...|[danny, cahill, s...|(262144,[571,991,...|
17291|First, a Mixtape....|[hillary, kerr, f...|(262144,[991,1998...|
17292|Calling on Angels...|[angels, everywhe...|(262144,[1712,236...|
17293|Weak Federal Powe...|[donald, trump, t...|(262144,[14,632,2...|
17294|Can Carbon Captur...|[thompsons, tex, ...|(262144,[14,329,5...|
17295|Mar-a-Lago, the F...|[west, palm, beac...|(262144,[408,1156...|
17296|How to form healt...|[article, part, s...|(262144,[14,2437,...|
17297|Turning Your Vaca...|[season, family, ...|(262144,[15,571,1...|
17298|As Second Avenue ...|[finally, second,...|(262144,[590,1232...|
17300|Dylann Roof Himse...|[pages, journal, ...|(262144,[14,619,8...|
17301|Modi’s Cash Ban B...|[mumbai, india, b...|(262144,[14,1461,...|
17302|Suicide Bombing i...|[baghdad, suicide...|(262144,[329,1728...|
17303|Fecal Pollution T...|[sydney, australi...|(262144,[560,2317...|
+-----+--------------------+--------------------+--------------------+
only showing top 20 rows

In [27]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [28]:
idf_model = idf.fit(featurized_data)

In [29]:
rescaled_data = idf_model.transform(featurized_data)

In [30]:
ii_df = rescaled_data.select("id","title","words","features")

In [31]:
ii_df.first()

Out[ 114 ]: Row(id='17283', title='House Republicans Fret About Winning Their Health Care Suit - The New York Times', words=['washington', 'congressional', 'republicans', 'new', 'fear', 'comes', 'health', 'care', 'lawsuit', 'obama', 'administration', 'might', 'win', 'incoming', 'trump', 'administration', 'choose', 'longer', 'defend', 'executive', 'branch', 'suit', 'challenges', 'administration', 'authority', 'spend', 'billions', 'dollars', 'health', 'insurance', 'subsidies', 'americans', 'handing', 'house', 'republicans', 'big', 'victory', 'issues', 'sudden', 'loss', 'disputed', 'subsidies', 'conceivably', 'cause', 'health', 'care', 'program', 'implode', 'leaving', 'millions', 'people', 'without', 'access', 'health', 'insurance', 'republicans', 'prepared', 'replacement', 'lead', 'chaos', 'insurance', 'market', 'spur', 'political', 'backlash', 'republicans', 'gain', 'full', 'control', 'government', 'stave', 'outcome', 'republicans', 'find', 'awkward', 'position', 'appropriating', 'huge', 'sums', 'temporarily', 'prop', 'obama', 'health', 'care', 'law', 'angering', 'conservative', 'voters', 'demanding', 'end', 'law', 'years', 'another', 'twist', 'donald', 'trump', 'administration', 'worried', 'preserving', 'executive', 'branch', 'prerogatives', 'choose', 'fight', 'republican', 'allies', 'house', 'central', 'questions', 'dispute', 'eager', 'avoid', 'ugly', 'political', 'pileup', 'republicans', 'capitol', 'hill', 'trump', 'transition', 'team', 'gaming', 'handle', 'lawsuit', 'election', 'put', 'limbo', 'least', 'late', 'february', 'united', 'states', 'court', 'appeals', 'district', 'columbia', 'circuit', 'yet', 'ready', 'divulge', 'strategy', 'given', 'pending', 'litigation', 'involves', 'obama', 'administration', 'congress', 'inappropriate', 'comment', 'said', 'phillip', 'blando', 'spokesman', 'trump', 'transition', 'effort', 'upon', 'taking', 'office', 'trump', 'administration', 'evaluate', 'case', 'related', 'aspects', 'affordable', 'care', 'act', 'potentially', 'decision', '2015', 'judge', 'rosemary', 'collyer', 'ruled', 'house', 'republicans', 'standing', 'sue', 'executive', 'branch', 'spending', 'dispute', 'obama', 'administration', 'distributing', 'health', 'insurance', 'subsidies', 'violation', 'constitution', 'without', 'approval', 'congress', 'justice', 'department', 'confident', 'judge', 'collyer', 'decision', 'reversed', 'quickly', 'appealed', 'subsidies', 'remained', 'place', 'appeal', 'successfully', 'seeking', 'temporary', 'halt', 'proceedings', 'mr', 'trump', 'won', 'house', 'republicans', 'last', 'month', 'told', 'court', 'transition', 'team', 'currently', 'discussing', 'potential', 'options', 'resolution', 'matter', 'take', 'effect', 'inauguration', 'jan', '20', '2017', 'suspension', 'case', 'house', 'lawyers', 'said', 'provide', 'future', 'administration', 'time', 'consider', 'whether', 'continue', 'prosecuting', 'otherwise', 'resolve', 'appeal', 'republican', 'leadership', 'officials', 'house', 'acknowledge', 'possibility', 'cascading', 'effects', 'payments', 'totaled', 'estimated', '13', 'billion', 'suddenly', 'stopped', 'insurers', 'receive', 'subsidies', 'exchange', 'paying', 'costs', 'deductibles', 'eligible', 'consumers', 'race', 'drop', 'coverage', 'since', 'losing', 'money', 'loss', 'subsidies', 'destabilize', 'entire', 'program', 'cause', 'lack', 'confidence', 'leads', 'insurers', 'seek', 'quick', 'exit', 'well', 'anticipating', 'trump', 'administration', 'might', 'inclined', 'mount', 'vigorous', 'fight', 'house', 'republicans', 'given', 'dim', 'view', 'health', 'care', 'law', 'team', 'lawyers', 'month', 'sought', 'intervene', 'case', 'behalf', 'two', 'participants', 'health', 'care', 'program', 'request', 'lawyers', 'predicted', 'deal', 'house', 'republicans', 'new', 'administration', 'dismiss', 'settle', 'case', 'produce', 'devastating', 'consequences', 'individuals', 'receive', 'reductions', 'well', 'nation', 'health', 'insurance', 'health', 'care', 'systems', 'generally', 'matter', 'happens', 'house',